In [ ]:
def getTides(prh, feed, datpth, tagonDS, tagoffDS):
    if not os.path.exists(datpth+depID+'\\'+depID+'_hilo.csv') and os.path.exists(datpth+depID+'\\'+depID+'_tides.csv'):
        startdt = dt.strptime(str(tagonDS), '%Y-%m-%d %H:%M:%S.%f').strftime('%Y-%m-%d')
        enddt = dt.strptime(str(tagoffDS), '%Y-%m-%d %H:%M:%S.%f').strftime('%Y-%m-%d')
        
        # NOAA Tides and Currents API endpoint
        api_url = 'https://tidesandcurrents.noaa.gov/api/datagetter'
        
        # Specify parameters for the API request
        station_id = '9447659'  # Everett
        product = 'predictions'
        begin_date = startdt.replace('-','')  # Replace with the start date in the format YYYYMMDD
        end_date = enddt.replace('-','')    # Replace with the end date in the format YYYYMMDD
        datum = 'MLLW'           # Mean Lower Low Water, other options include 'NAVD' and 'IGLD'
        interval1 = 1             # one minute tide predictions
        interval2 = 'hilo'

        # Make the API request
        response1 = requests.get(api_url, params={
            'station': station_id,
            'product': product,
            'begin_date': begin_date,
            'end_date': end_date,
            'datum': datum,
            'units': 'english',
            'time_zone': 'gmt',
            'format': 'json',
            'interval':interval1
        })

        response2 = requests.get(api_url, params={
            'station': station_id,
            'product': product,
            'begin_date': begin_date,
            'end_date': end_date,
            'datum': datum,
            'units': 'english',
            'time_zone': 'gmt',
            'format': 'json',
            'interval':interval2
        })
        
        # Check if the request was successful
        if response1.status_code == 200:
            tidal_data = response1.json()
            tides = pd.DataFrame(tidal_data['predictions'])
            hilo_data = response2.json()
            hilotides = pd.DataFrame(hilo_data['predictions'])

    
        else:
            print(f"Failed to retrieve data. Status Code: {response.status_code}")

        tides['t'] = pd.to_datetime(tides['t'])
        tides['Date Time'] = pd.to_datetime(tides['Date Time'])
        tides.to_csv(datpth+depID+'\\'+depID+'_tides.csv')
        hilotides['t'] = pd.to_datetime(hilo['t'])
        hilotides['Date Time'] = pd.to_datetime(hilotides['Date Time'])
        hilotides.to_csv(datpth+depID+'\\'+depID+'_hilo.csv')
        # Create deployment only tidal structure and wrangle dataframes
        depTides = pd.DataFrame(tides[(tides['Date Time']>= tagonDS) & (tides['Date Time'] <= tagoffDS)]) # subset to only deployment tidal values
        depTides['indTides'] = depTides.index
        depTides = depTides.reset_index(drop=True)
        depTides.columns = ['DateTime', 'Prediction', 'indTides']
        print('Saved tidal data to data dir...')
        return(tides,hilotides, depTides)

    else:
        print('Loading tidal data...')
        
        tides = pd.DataFrame(pd.read_csv(datpth+depID+'\\'+depID+'_tides.csv'))
        tides['Date Time'] = pd.to_datetime(tides['Date Time'])

        hilotides = pd.DataFrame(pd.read_csv(datpth+depID+'\\'+depID+'_hilo.csv'))
        #tides['t'] = pd.to_datetime(tides['t'])
        #hilotides['t'] = pd.to_datetime(hilo['t'])
        # Create deployment only tidal structure and wrangle dataframes
        depTides = pd.DataFrame(tides[(tides['Date Time']>= tagonDS) & (tides['Date Time'] <= tagoffDS)]) # subset to only deployment tidal values
        depTides['indTides'] = depTides.index
        depTides = depTides.reset_index(drop=True)
        depTides.columns = ['DateTime', 'Prediction', 'indTides']
        return(tides, hilotides, depTides)